# 进化算法的基本操作与实现
学习进化算法的笔记第一弹

## 参考资料
+ [基于DEAP库的Python进化算法从入门到入土--(一)进化算法的基本操作与实现](https://www.jianshu.com/p/8fa044ed9267)
+ deap源码与文档

## 遗传算法的优缺点
优点：

+ 泛用性强，对连续变量和离散变量都能适用；
+ 不需要导数信息，因此不要求适应度函数的连续和可微性质(或者说不需要问题内在机理的相关信息)；
+ 可以在解空间内大范围并行搜索；
+ 不容易陷入局部最优；
+ 高度并行化，并且容易与其他优化方法整合。

缺点：

+ 对于凸优化问题，相对基于梯度的优化方法（例如梯度下降法，牛顿/拟牛顿法）收敛速度更慢；
+ 进化算法需要在搜索空间投放大量个体来搜索最优解。对于高维问题，由于搜索空间随维度指数级膨胀，需要投放的个体数也大幅增长，会导致收敛速度变慢；
+ 设计编码方式、适应度函数以及变异规则需要大量经验。

## 问题定义、个体编码与创建初始族群
### 1.优化问题的定义：通过定义`base.Fitness`的子类来描述适应度这个指标
单目标优化：`creator.create('FitnessMin', base.Fitness, weights=(-1.0, ))`

在创建单目标优化问题时，weights用来指示最大化和最小化。此处-1.0即代表问题是一个最小化问题，对于最大化，应将weights改为正数，如1.0。

另外即使是单目标优化，weights也需要是一个tuple，以保证单目标和多目标优化时数据结构的统一。

对于单目标优化问题，weights 的绝对值没有意义，只要符号选择正确即可。

多目标优化：`creator.create('FitnessMulti', base.Fitness, weights=(-1.0, 1.0))`

对于多目标优化问题，weights用来指示多个优化目标之间的相对重要程度以及最大化最小化。如示例中给出的(-1.0, 1.0)代表对第一个目标函数取最小值，对第二个目标函数取最大值。

#### 补充内容： `creator.create(name, base, **kargs)` 

调用该函数后，会在creator模块中以base类为基类，设置kwargs中的属性而生成一个子类，该子类可以通过creator.name来访问。如果kargs传入的是类型，则会在实例化子类时自动调用该类型的无参构造器而给该实例创建一个成员变量；而当kargs传入对象时，则会将该对象作为这个子类的静态成员。

```python
create("Foo", list, bar=dict, spam=1)

# 等价于在creator模块中加入了以下定义

class Foo(list):
    spam = 1

    def __init__(self):
        self.bar = dict()
```

#### 补充内容：`base.Fitness`
用于比较结果好坏的一个指标。Fitness拥有value与weights两个属性，应当有相同的size。Fitness能够使用> < >= <= == != 等算符来进行比较value与weights两个向量的乘积（因此能够解释上文weights的设置原因），通过字典序进行判定。两个长度不同的Fitness能够进行比较，如果短的部分相同的话，长的那个更优。

### 2.个体编码
**实数编码(Value encoding)**：

直接用实数对变量进行编码。优点是不用解码，基因表达非常简洁，而且能对应连续区间。但是实数编码后搜索区间连续，因此容易陷入局部最优。

In [1]:
"""
实数编码的DEAP实现。生成一个有五个染色体的个体
"""
from deap import base, creator, tools
import random
IND_SIZE = 5
creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) #优化目标：单变量，求最小值
creator.create('Individual', list, fitness = creator.FitnessMin) #创建Individual类，继承list

toolbox = base.Toolbox()
toolbox.register('Attr_float', random.random)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Attr_float, n=IND_SIZE)

ind1 = toolbox.Individual()
ind1

[0.015262619864482518,
 0.8212760846404132,
 0.0241824946082011,
 0.4236115471530866,
 0.14164400911705144]

#### 补充内容：`toolbox.register(alias, function, *args, **kargs)`
在toolbox对象中注册一个函数alias，基于function并会自动填入args与kargs的参数。缺省参数在调用时提供。
```shell
>>> def func(a, b, c=3):
...     print(a, b, c)
...
>>> tools = Toolbox()
>>> tools.register("myFunc", func, 2, c=4)
>>> tools.myFunc(3)
2 3 4
```

#### 补充内容：`tools.initRepeat(container: function, func: function, n)`
实例化一个container对象，调用n次func，将结果置入container，然后返回container实例。

相当于 `return container(func(i) for in in range(n))`

练习：改为使用numpy生成相同的个体

In [2]:
import numpy as np
IND_SIZE = 5
creator.create('IndividualWithNp', np.ndarray, fitness = creator.FitnessMin)

toolbox1 = base.Toolbox()
toolbox1.register('attr_float', np.random.rand)
toolbox1.register('get_individual', tools.initRepeat, creator.IndividualWithNp, toolbox1.attr_float, 5)
ind1 = toolbox1.get_individual()
ind1

IndividualWithNp([0.77975773, 0.35820208, 0.51549664, 0.22248106,
                  0.05506382])

#### 二进制编码(Binary encoding)：
在二进制编码中，用01两种数字模拟人类染色体中的4中碱基，用一定长度的01字符串来描述变量。其优点在于种群多样性大，但是需要解码，而且不连续，容易产生Hamming cliff（例如0111=7, 1000=8，改动了全部的4位数字之后，实际值只变动了1），在接近局部最优位置时，染色体稍有变动，就会使变量产生很大偏移（格雷编码（Gray coding）能够克服汉明距离的问题，但是实际问题复杂度较大时，格雷编码很难精确描述问题）。

**变量的二进制编码：**

由于通常情况下，搜索空间都是实数空间，因此在编码时，需要建立实数空间到二进制编码空间的映射。使用二进制不能对实数空间进行连续编码，但是可以在给定精度下对连续空间进行离散化。

以例子来说明如何对变量进行二进制编码，假设需要对一个在区间$[-2, 2]$上的变量进行二进制编码：

*选择编码长度*：在需要6位精度的情况下，我们需要将该区间离散为$(2+2)*10^6$个数。由于$2^{22}>4*10^6$，我们至少需要22位二进制数字来满足我们的精度要求。

*设置解码器*：将二进制数字$x^{bin}$转化为十进制$x^{dec}$之后（在python中可以用`int('Binary number', 2)`来实现），按照公式$x=-2+x^{dec}*4/(2^{22}-1)$，-1以得到一个在$[-2, 2]$区间内的实数。

*实现*：根据上面的实数编码不难发现，deap的个体概念与其所采用的底层数据存储无关（上面的示例与练习都分别采用了`list`与`np.ndarray`），所以各种个体编码本质上实现方式相同，均选择合适的数据结构进行存储即可。下面的示例作者依旧使用了list进行存储，没有契合二进制存储的特征（选择的数据结构应当支持`[]`运算符），有优化空间。这个示例主要是展示如何选择随机初始化函数。


In [4]:
"""
二进制编码DEAP实现：

以随机生成一个长度为10的二进制编码为例，本身DEAP库中没有内置的Binary encoding，我们可以借助Scipy模块中的伯努利分布来生成一个二进制序列。
"""
from scipy.stats import bernoulli

creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) #优化目标：单变量，求最小值
creator.create('Individual', list, fitness = creator.FitnessMin) #创建Individual类，继承list

GENE_LENGTH = 10

toolbox = base.Toolbox()
toolbox.register('Binary', bernoulli.rvs, 0.5) #注册一个Binary的alias，指向scipy.stats中的bernoulli.rvs，概率为0.5
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Binary, n = GENE_LENGTH) #用tools.initRepeat生成长度为GENE_LENGTH的Individual

ind1 = toolbox.Individual()
ind1

/home/airan/miniconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/airan/miniconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


[1, 0, 0, 1, 1, 0, 0, 0, 1, 1]

#### 序列编码(Permutation encoding)：
通常在求解顺序问题时用到，例如TSP问题。序列编码中的每个染色体都是一个序列。
> 存在一个客观的序列，而染色体的值是查该序列的下标，整条染色体是这个原始序列的一个排列
> 下面的示例中，该序列就是`range(10)`

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("Indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("Individual", tools.initIterate, creator.Individual,toolbox.Indices)
ind1 = toolbox.Individual()
ind1

/home/airan/miniconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/airan/miniconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


[3, 6, 0, 9, 1, 2, 8, 4, 7, 5]

#### 补充内容：`np.random.permutation(x)`
重载函数。如果x是一个uint，那么会返回长度为x的一个乱序ndarray，等同于上面的例子中长度为10的染色体序列。而如果x是一个序列，则会直接打乱这个序列返回ndarray。

#### 补充内容：`initIterate: (container: function, generator: function)`
generator需要返回一个Iterable，initIterate相当于`return container(generator())`

#### 粒子(Particles)：
粒子是一种特殊个体，主要用于粒子群算法。相比普通的个体，它额外具有速度、速度限制并且能记录最优位置。

> TODO暂时跳过这部分，等之后学习到粒子群算法再返回

In [ ]:
"""
粒子的DEAP实现
"""
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=None,
               smin=None, smax=None, best=None)

# 自定义的粒子初始化函数
def initParticle(pcls, size, pmin, pmax, smin, smax):
    part = pcls(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

toolbox = base.Toolbox()
toolbox.register("Particle", initParticle, creator.Particle, size=2, pmin=-6, pmax=6, smin=-3, smax=3) #为自己编写的initParticle函数注册一个alias "Particle"，调用时生成一个2维粒子，放在容器creator.Particle中，粒子的位置落在（-6,6）中，速度限制为（-3，3）

ind1 = toolbox.Particle()
print(ind1)
print(ind1.speed)
print(ind1.smin, ind1.smax)

# 结果：[-2.176528549934324, -3.0796558214905]
#[-2.9943676285620104, -0.3222138308543414]
#-3 3

print(ind1.fitness.valid)

# 结果：False
# 因为当前还没有计算适应度函数，所以粒子的最优适应度值还是invalid

### 3.初始族群的创建
#### 一般族群：
这是最常用的族群类型，族群中没有特别的顺序或者子族群。

一般族群的DEAP实现：`toolbox.register('population', tools.initRepeat, list, toolbox.individual)`

In [11]:
"""
以二进制编码为例，以下代码可以生成由10个长度为5的随机二进制编码个体组成的一般族群
"""
# 定义问题
creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) # 单目标，最小化
creator.create('Individual', list, fitness = creator.FitnessMin)

# 生成个体
GENE_LENGTH = 5
toolbox = base.Toolbox() #实例化一个Toolbox
toolbox.register('Binary', bernoulli.rvs, 0.5)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Binary, n=GENE_LENGTH)

# 生成初始族群
N_POP = 10
toolbox.register('Population', tools.initRepeat, list, toolbox.Individual) # 这里注册时缺省了initRepeat函数的“n”参数，而在实际调用时提供。
toolbox.Population(n = N_POP)

/home/airan/miniconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/airan/miniconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


[[1, 1, 1, 0, 1],
 [0, 1, 1, 0, 1],
 [1, 1, 1, 1, 0],
 [1, 0, 1, 1, 0],
 [1, 0, 1, 0, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 0, 1],
 [0, 0, 0, 1, 1],
 [0, 1, 0, 1, 0],
 [1, 1, 0, 1, 1]]

#### 同类群(Demes)：
同类群即一个族群中包含几个子族群。在有些算法中，会使用本地选择(Local selection)挑选育种个体，这种情况下个体仅与同一邻域的个体相互作用。

In [15]:
"""
同类群的DEAP实现：其实就是几个一般群的数组
"""
toolbox.register("deme", tools.initRepeat, list, toolbox.Individual) # 与一般群相同

DEME_SIZES = 10, 50, 100
population = [toolbox.deme(n=i) for i in DEME_SIZES] # 太长了不打印了

#### 粒子群(Swarm)：
粒子群中的所有粒子共享全局最优。在实现时需要额外传入全局最优位置与全局最优适应度给族群。

粒子群的DEAP实现：

```python
# 其实就是扩展了用于存储群的list，加入了全局最优位置和全局最优适应度两个属性。
creator.create("Swarm", list, gbest=None, gbestfit=creator.FitnessMax)
toolbox.register("swarm", tools.initRepeat, creator.Swarm, toolbox.particle)
```

在算法迭代时，需要更新该轮迭代中最优的位置和最优适应度。

## 评价

## 配种选择

## 变异

## 环境选择